In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd 
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from torchvision.datasets import ImageFolder
import torch.optim as optim
import torch.nn as nn

# Ignore warnings 
import warnings 
warnings.filterwarnings('ignore') 

# Import pathlib
from pathlib import Path

# Interactive mode on 
plt.ion() 

In [2]:
from torch import cuda

cuda.set_device(0)
current_dev = cuda.current_device()
current_dev_name = cuda.get_device_name(current_dev)
current_dev_specs = cuda.get_device_properties(current_dev)

print(f'Current Device: {current_dev}')
print(f'Current Device Name: {current_dev_name}')
print(f'Current Device Specs: {current_dev_specs}')

Current Device: 0
Current Device Name: TITAN V
Current Device Specs: _CudaDeviceProperties(name='TITAN V', major=7, minor=0, total_memory=12066MB, multi_processor_count=80)


In [63]:
mobilenet = torch.hub.load('pytorch/vision:v0.4.2', 'mobilenet_v2', pretrained=True)
mobilenet.classifier[1] = nn.Linear(1280, 2)
mobilenet.to('cuda')

Using cache found in /home/SGF.EDUBEAR.NET/eam96/.cache/torch/hub/pytorch_vision_v0.4.2


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [64]:
# Create data for model
path = Path.cwd().parents[0] / 'data' / 'heat_maps'

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img_set = ImageFolder(path, transform=preprocess)
trainloader = DataLoader(img_set, batch_size=4, num_workers=4, shuffle=True)

In [65]:
# MobileNet hyperparameters 
num_epochs = 1
eta = 0.001
nu = 0.9

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=eta)

In [66]:
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to('cuda'), data[1].to('cuda')

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mobilenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.335
[1,    20] loss: 0.172
[1,    30] loss: 0.019
[1,    40] loss: 0.003
Finished Training


In [67]:
# Create data for model
path = Path.cwd().parents[0] / 'data' / 'heat_maps'

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img_set = ImageFolder(path, transform=preprocess)
testloader = DataLoader(img_set, batch_size=len(img_set), num_workers=4, shuffle=True)

In [68]:
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data[0].to('cuda'), data[1].to('cuda') 

        outputs = mobilenet(inputs)

        correct, total = 0, outputs.shape[0]
        for output, label in zip(outputs, labels):
            hypothesis = output.max(0)[1].item()
            print(f'Output: {hypothesis} \tExpected: {label}')
            
            if hypothesis == label: correct += 1
                
        print(f'Accuracy on test data: {(correct/total)*100}')           

Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	

In [69]:
torch.save(mobilenet.state_dict(), '/home/SGF.EDUBEAR.NET/eam96/Net_Sentinel/data/models/mobile_net.pth')

/home/SGF.EDUBEAR.NET/eam96/Net_Sentinel/nbs


In [57]:
squeezenet = torch.hub.load('pytorch/vision:v0.4.2', 'squeezenet1_1', pretrained=True)
squeezenet.classifier[1] = nn.Conv2d(512, 2, kernel_size=(1,1), stride=(1,1))
squeezenet.to('cuda')

Using cache found in /home/SGF.EDUBEAR.NET/eam96/.cache/torch/hub/pytorch_vision_v0.4.2


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [58]:
# MobileNet hyperparameters 
num_epochs = 1
eta = 0.001
nu = 0.9

criterion2 = nn.CrossEntropyLoss()
optimizer2 = optim.Adam(squeezenet.parameters(), lr=eta)

In [59]:
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to('cuda'), data[1].to('cuda')

        # zero the parameter gradients
        optimizer2.zero_grad()

        # forward + backward + optimize
        outputs = squeezenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer2.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.993
[1,    20] loss: 0.329
[1,    30] loss: 0.050
[1,    40] loss: 0.006
Finished Training


In [61]:
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data[0].to('cuda'), data[1].to('cuda') 

        outputs = squeezenet(inputs)

        correct, total = 0, outputs.shape[0]
        for output, label in zip(outputs, labels):
            hypothesis = output.max(0)[1].item()
            print(f'Output: {hypothesis} \tExpected: {label}')
            
            if hypothesis == label: correct += 1
                
        print(f'Accuracy on test data: {(correct/total)*100}')  

Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 1 	Expected: 1
Output: 0 	Expected: 0
Output: 0 	Expected: 0
Output: 0 	

In [70]:
torch.save(squeezenet.state_dict(), '/home/SGF.EDUBEAR.NET/eam96/Net_Sentinel/data/models/squeeze_net.pth')